In [1]:
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [18]:
def prepare_df(df):
    df["topic"] = df[df["text"].isin(topic)]

    df[["speaker", 'text']] = df["text"].str.split(":", expand=True)
    df["topic"] = df["topic"].fillna(method="ffill")
    df = df[df["speaker"].str.contains("P")]
    df = df[['speaker', 'topic', 'text']].reset_index(drop=True)
    
    return df.dropna()


def read_text():
    return pd.read_table("data/session-1-transcribed-annotated.txt",
                         header=None,
                         names=["text"]) 

def strip_spaces(df):
    df['text'] = df["text"].str.strip()
    return df

In [19]:
topic = ["USAGE", "TECHNOLOGY", "EMOTION", "COMFORT", "EXPERIENCE", "SATISFACTION"]

df = read_text()
df = prepare_df(df)
df = strip_spaces(df)

In [14]:
#clean the data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [21]:
df["clean_text"] = df['text'].apply(clean)

In [22]:
df

,speaker,topic,text,clean_text
0,P*,USAGE,Mainly groups working together as it's an open...,"[mainly, group, working, together, open, envir..."
1,P*,USAGE,I also like quite a non-stressy work. Like jus...,"[also, like, quite, nonstressy, work, like, ca..."
2,P,USAGE,You could also use it for a lunch or something.,"[could, also, use, lunch, something]"
3,P,TECHNOLOGY,How smart do you think this place is? How do y...,"[smart, think, place, is, think, technology, i..."
4,P,TECHNOLOGY,"It's not really smart, I think. There are some...","[really, smart, think, socket, know, corner, s..."
...,...,...,...,...
131,P*,SATISFACTION,Cottage like. Cottage like. Which is cozy. But...,"[cottage, like, cottage, like, cozy, like, spe..."
132,P*,SATISFACTION,"I feel at peace, this space is I know Roomy.","[feel, peace, space, know, roomy]"
133,P,SATISFACTION,For me it's not comfortable so… uncomfortable.,"[comfortable, so…, uncomfortable]"
134,P,SATISFACTION,"Yeah, I just …the light…[from the artificial s...","[yeah, …the, light…from, artificial, spotlight..."


In [26]:
def create_dictionary(df):
    dictionary = corpora.Dictionary(df['clean_text'])
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['clean_text'] ]
    
    print("Total no of Non Zeros in BOW:", dictionary.num_nnz)
    print("Length of document matrix:", len(doc_term_matrix))

    return doc_term_matrix

In [27]:
#create dictionary
doc_term_matrix = create_dictionary(df)

Total no of Non Zeros in BOW: 1578
Length of document matrix: 130
